In [1]:
import json
import os

In [4]:
root = 'C:/tmp/mlp/mlp-output/'

titles = []
for f in os.listdir(root): 
    for line in file(root + f):
        doc = json.loads(line)

        title = doc['title']
        if '(disambiguation)' in title:
            continue

        titles.append(title)

In [5]:
titles_set = {doc for doc in titles}

In [6]:
prefix_len = len('<http://dbpedia.org/resource/')
prefix_cat_len = len('<http://dbpedia.org/resource/Category:')

def split_ttl(ttl):
    split = ttl.split()
    page = split[0][prefix_len:-1]
    cat = split[2][prefix_cat_len:-1]
    return page.replace('_', ' ').decode('utf-8'), cat.replace('_', ' ').decode('utf-8')

In [7]:
ttl_categories = file('C:/tmp/mlp/article_categories_en.ttl')

from collections import defaultdict
ttl_titles = defaultdict(set)
ttl_cats = defaultdict(set)

for line in ttl_categories:
    if line.startswith('#'):
        continue
    page, cat = split_ttl(line)
    if page in titles_set:
        ttl_titles[page].add(cat)
        ttl_cats[cat].add(page)

KeyboardInterrupt: 

In [ ]:
len(ttl_titles), len(ttl_cats)

In [30]:
shared = titles_set.intersection(ttl_titles.keys())
len(shared), len(titles_set)

(28364, 29977)

In [31]:
not_found = titles_set.difference(shared)

In [32]:
len(not_found)

1613

In [33]:
import urllib

In [34]:
downloaded = {}

In [36]:
for idx, title in enumerate(not_found):
    # http://dbpedia.org/data/Function_of_several_real_variables.ntriples
    url = 'http://dbpedia.org/data/' +  title.replace(' ', '_').encode('utf-8') + '.ntriples'
    f = urllib.urlopen(url)
    myfile = f.read()
    downloaded[title] = myfile
    
    if idx % 100 == 0:
        print 'progress: %d out of %d...' % (idx, len(not_found))

progress: 0 out of 1613...
progress: 100 out of 1613...
progress: 200 out of 1613...
progress: 300 out of 1613...
progress: 400 out of 1613...
progress: 500 out of 1613...
progress: 600 out of 1613...
progress: 700 out of 1613...
progress: 800 out of 1613...
progress: 900 out of 1613...
progress: 1000 out of 1613...
progress: 1100 out of 1613...
progress: 1200 out of 1613...
progress: 1300 out of 1613...
progress: 1400 out of 1613...
progress: 1500 out of 1613...
progress: 1600 out of 1613...


In [37]:
len(downloaded)

1613

In [38]:
predicate = '<http://purl.org/dc/terms/subject>'

for title, content in downloaded.items():
    for line in content.split('\n'):
        if predicate not in line:
            continue

        _, cat = split_ttl(line)

        ttl_titles[title].add(cat)
        ttl_cats[cat].add(title)

In [39]:
shared = titles_set.intersection(ttl_titles.keys())
len(shared), len(titles_set)

(29764, 29977)

In [47]:
not_found = titles_set.difference(shared)
len(not_found)

213

In [12]:
import codecs

In [22]:
category_file = codecs.open('C:/tmp/mlp/category_info.txt', 'w', encoding='utf-8')

for title, cats in ttl_titles.items():
    for cat in cats:
        category_file.write(u'%s\t%s\n' % (title, cat))
        
category_file.close()

NameError: name 'ttl_titles' is not defined

This was used to process categories:

**read previously processed categories**

    from collections import defaultdict

    doc_categories = defaultdict(set)
    category_docs = defaultdict(set)

    category_blacklist = {u'Articles containing proofs', 
                          u'Articles created via the Article Wizard', 
                          u'Articles with example pseudocode'}

    for line in file('C:/tmp/mlp/category_info.txt'):
        title, cat = line.strip().split('\t')
        title = title.decode('utf-8')
        cat = cat.decode('utf-8')

        if cat in category_blacklist:
            continue

        doc_categories[title].add(cat) 
        category_docs[cat].add(title)

    print len(doc_categories), len(category_docs)

    small_cats = set()

    for cat, docs in category_docs.items():
        if len(docs) == 1:
            small_cats.add(cat)

    print len(small_cats)

    for cat in small_cats:
        for doc in category_docs[cat]:
            doc_categories[doc].remove(cat)
        del category_docs[cat]

    del small_cats

    for doc in doc_categories.keys():
        if len(doc_categories[doc]) == 0:
            doc_categories[doc].add(u'OTHER')
        category_docs[u'OTHER'].add(doc)



**read documens, discard empty ones**

    root = 'C:/tmp/mlp/mlp-output/'

    docs = []
    titles = []
    ids = []
    rels = []

    empty = 0
    small = 0

    for f in os.listdir(root): 
        for line in file(root + f):
            doc = json.loads(line)

            title = doc['title']        
            if title not in doc_categories:
                continue

            if '(disambiguation)' in title:
                continue

            id_bag = id_counter(doc['identifiers'])
            if len(id_bag) <= 1:
                if len(id_bag) == 0:
                    empty = empty + 1
                else:
                    small = small + 1
                continue

            docs.append(doc)
            titles.append(title)
            ids.append(id_bag)

            id_rels = rel_to_dict(doc['relations'])
            rels.append(id_rels)


**then we remove all categories that aren't used in the documents**

    title_idx = {title: idx for (idx, title) in enumerate(titles)}

    for doc, cats in doc_categories.items():
        if doc in title_idx:
            continue

        for cat in cats: 
            category_docs[cat].remove(doc)

        del doc_categories[doc]

    print len(doc_categories)

    title_cats = [doc_categories[title] for title in titles]
    doc_cat_flat = {}

    for line in file('C:/tmp/mlp/category_info.txt'):
        title, cat = line.strip().split('\t')
        title = title.decode('utf-8')
        cat = cat.decode('utf-8')
        doc_cat_flat[title] = cat

    title_cats_flat = [doc_cat_flat[title] for title in titles]
    del doc_cat_flat
    print len(title_cats_flat)

    all_categories = list(set(title_cats_flat))
    categories_idx = {cat: idx for idx, cat in enumerate(all_categories)}
    title_cats_code = np.array([categories_idx[cat] for cat in title_cats_flat])


Now let's add "broader" categories

In [12]:
category_file = file('C:/tmp/mlp/category_info.txt')

math_categories = set()

document_categories = defaultdict(set)

for line in category_file:
    title, cat = line.strip().split('\t')
    title = title.decode('utf-8')
    cat = cat.decode('utf-8')

    document_categories[title].add(cat)
    math_categories.add(cat)
    
category_file.close()

In [13]:
len(math_categories)

4798

In [14]:
len(document_categories)

22822

In [15]:
import numpy as np

In [16]:
np.mean([len(cat) for _, cat in document_categories.items()])

2.226316711944615

In [17]:
prefix_cat_len = len('<http://dbpedia.org/resource/Category:')

In [18]:
skos_broader_predicate = '<http://www.w3.org/2004/02/skos/core#broader>'
skos_categories_file = file('C:/tmp/mlp/skos_categories_en.ttl')

from collections import defaultdict
skos_broader = defaultdict(set)
                       
for line in skos_categories_file:
    if line.startswith('#'):
        continue
    line = line.strip()
    split = line.split()
    predicate = split[1].strip()
    
    if predicate != skos_broader_predicate:
        continue

    cat = split[0][prefix_cat_len:-1].replace('_', ' ').decode('utf-8')
    if cat not in math_categories:
        continue

    broader_cat = split[2][prefix_cat_len:-1].replace('_', ' ').decode('utf-8')    
    skos_broader[cat].add(broader_cat)

skos_categories_file.close()

In [19]:
len(skos_broader)

4577

In [20]:
skos_broader['Numerical linear algebra']

{u'Linear algebra', u'Numerical analysis'}

In [45]:
for doc, categories in document_categories.items():
    broader_cats = set()
    for cat in categories:
        broader_cats.update(skos_broader[cat])
    
    categories.update(broader_cats)

In [46]:
np.mean([len(cat) for _, cat in document_categories.items()])

7.6373674524581547

In [48]:
document_categories['Principal component analysis']

{u'Analysis',
 u'Data analysis',
 u'Data management',
 u'Data mining',
 u'Dimension',
 u'Dimension reduction',
 u'Machine learning',
 u'Matrix decompositions',
 u'Matrix theory',
 u'Multivariate statistics',
 u'Numerical linear algebra',
 u'Statistical data types',
 u'Statistics'}

In [51]:
from collections import Counter

In [53]:
cat_df = Counter()

for doc, categories in document_categories.items():
    cat_df.update(categories)


cat_df.most_common(100)

[(u'Mathematical analysis', 1981),
 (u'Physics', 1596),
 (u'Abstract algebra', 1517),
 (u'Theoretical physics', 1479),
 (u'Fields of mathematics', 1235),
 (u'Geometry', 1079),
 (u'Statistics', 1049),
 (u'Quantum mechanics', 973),
 (u'Mechanics', 972),
 (u'Functions and mappings', 956),
 (u'Algorithms', 795),
 (u'Differential geometry', 793),
 (u'Numerical analysis', 792),
 (u'Combinatorics', 781),
 (u'Applied mathematics', 772),
 (u'Particle physics', 748),
 (u'Mathematical theorems', 746),
 (u'Number theory', 741),
 (u'Algebra', 732),
 (u'Dynamical systems', 706),
 (u'Applied and interdisciplinary physics', 700),
 (u'Fields of application of statistics', 676),
 (u'Chemistry', 670),
 (u'Topology', 664),
 (u'Linear algebra', 662),
 (u'Physical chemistry', 647),
 (u'Probability theory', 607),
 (u'Fluid dynamics', 601),
 (u'Manifolds', 594),
 (u'Differential equations', 591),
 (u'Materials science', 587),
 (u'Mathematical physics', 583),
 (u'Group theory', 577),
 (u'Statistical data types

In [70]:
singleton_categories = set(cat for cat, cnt in cat_df.items() if cnt == 1)

In [71]:
len(singleton_categories)

793

In [60]:
for doc, categories in document_categories.items():
    categories.difference_update(singleton_categories)

In [69]:
empty = 0
docs = []
for doc, categories in document_categories.items():
    if not categories:
        empty = empty + 1
        docs.append(doc)

print docs

[]


In [67]:
for doc, categories in document_categories.items():
    if not categories:
        categories.add(u'OTHER')

In [69]:
empty = 0
docs = []
for doc, categories in document_categories.items():
    if not categories:
        empty = empty + 1
        docs.append(doc)

print docs

[]


In [68]:
category_file = codecs.open('C:/tmp/mlp/category_info_refined.txt', 'w', encoding='utf-8')

for doc, categories in document_categories.items():
    for cat in categories:
        category_file.write(u'%s\t%s\n' % (doc, cat))
        
category_file.close()

### Now let's create a tree of categories

In [32]:
import codecs
skos_cats_file = codecs.open('C:/tmp/mlp/skos_math_broader.txt', 'w', encoding='utf-8')

for k, v in skos_broader.items():
    skos_cats_file.write('\t'.join([k] + list(v)))
    skos_cats_file.write('\n')

skos_cats_file.close()    

In [31]:
skos_broader['Statistics']

{u'Mathematical and quantitative methods (economics)',
 u'Mathematical sciences',
 u'Probability and statistics'}